In [1]:
import numpy as np
import pandas as pd
import bayes_logistic as bl
import time
from sklearn.linear_model import LogisticRegression as lr
import matplotlib.pyplot as plt

In [2]:
def fixedexpress(mean,cov,resp,design):
    numitems=len(mean)
    N=np.max(design)+1
    items=np.random.choice([x for x in range(0,numitems)],N,False)
    return items[design]

In [3]:
def fixedsparse(mean,cov,resp,design):
    numitems=len(mean)
    if resp==1:
        order=[]
        for i in range(775):
            val=[x for x in range(numitems)]
            np.random.shuffle(val)
            order+=val
        np.save('fixed'+str(numitems)+'.npy',order)
    else:
        order=np.load('fixed'+str(numitems)+'.npy')
    N=design.size
    items=order[resp*N:(resp+1)*N]
    return np.reshape(items,design.shape)

In [4]:
#results=pd.DataFrame

In [5]:
#results.columns=["RespIDNum","QNum","Choice1","Choice2","Choice3","Choice4","Choice5",
#                 "Best","Worst","BestInd","WorstInd"]

In [4]:
def simulation(nrespondents,samplescheme,batchsize,itemfile,designfile,seed=0):
    if seed>0:
        np.random.seed(seed)#for reproducibility
    data=pd.read_csv(itemfile)#
    MDDesign =pd.read_csv(designfile)#get the files
    utilsall=data.get_values()[:,2:]#utils of bots
    [numreal,nitems]=utilsall.shape#
    design=MDDesign.get_values()[:,2:]#changes to a numpy array
    numver=max(MDDesign['Version'])#number of versions
    qperset=max(MDDesign['Set'])#questions per set
    itemsperq=design.shape[1]#items per questions
    itemsperset=np.max(design)#items per set
    people=np.random.choice([x for x in range(0,numreal)],nrespondents)#choice the bots
    cov=np.eye(nitems)
    totalrows=qperset*itemsperq#rows
    nb=np.zeros(nitems)
    nw=np.zeros(nitems)
    allquestions=np.zeros((nrespondents*qperset,itemsperq),dtype=np.int)
    allbest=np.zeros(nrespondents*qperset,dtype=np.int)
    allworst=np.zeros(nrespondents*qperset,dtype=np.int)
    X=np.zeros((nrespondents*qperset,nitems))
    w=np.zeros(nitems)
    numrows=nrespondents//batchsize
    betas=np.zeros((numrows,nitems))
    for iternum in range(0,nrespondents):
        util=utilsall[people[iternum],:]
        vernum=iternum%numver
        respdesign=design[qperset*vernum:(qperset)*(vernum+1)]-1
        questions=samplescheme(w,cov,iternum,respdesign)#gets items
        bestitems,worstitems=fillsurvey(questions,util)
        for i,bitem in enumerate(bestitems):
            nb[bitem]+=1
            allbest[iternum*qperset+i]=bitem
        for i,witem in enumerate(worstitems):
            nw[witem]+=1
            allworst[iternum*qperset+i]=witem
        allquestions[iternum*qperset:(iternum+1)*qperset]=questions
        for i,oneq in enumerate(questions):
            X[iternum*qperset+i,oneq]=1
        if iternum%batchsize==(batchsize-1):
            #return X[:(iternum+1)*qperset]
            w,cov=estimateparams(X[:(iternum+1)*qperset],
                             allquestions[:(iternum+1)*qperset],nb,nw,w)
            betas[(iternum+1)//(batchsize)-1]=w
    return X,allquestions,allbest,allworst
    return betas,allquestions,allbest,allworst,people

In [5]:
def fillsurvey(questions,util):
    qperset,itemsperq=questions.shape
    noiseb= -np.log(-np.log(np.random.rand(qperset,itemsperq)))
    noisew=  np.log(-np.log(np.random.rand(qperset,itemsperq)))
    combutilb=util[questions]+noiseb
    combutilw=util[questions]+noisew
    #indexb=np.argmax(combutilb,1)
    #indexw=np.argmin(combutilw,1)
    best=questions[range(qperset),np.argmax(combutilb,1)]
    worst=questions[range(qperset),np.argmin(combutilw,1)]
    return best,worst

In [6]:
def updatedata(questions,indexb,indexw,totalrows,nitems):
    qperset,itemsperq=questions.shape
    indexofq=questions.flatten()
    Xb=np.zeros((totalrows,nitems))
    Xb[range(0,len(indexofq)),indexofq]=1
    #Xb[indexofq==(nitems-1)]=1
    X=np.zeros((totalrows*2,nitems))
    best=[x for x in range(0,totalrows*2) if x//itemsperq%2 == 0]
    worst=[x for x in range(0,totalrows*2) if x//itemsperq%2 == 1]
    X[best]=Xb[:,:]
    X[worst]=-Xb[:,:]
    yb=np.zeros(totalrows)
    yw=np.zeros(totalrows)
    stride=np.array([x for x in range(0,totalrows,5)])
    yb[stride+indexb]=1
    yw[stride+indexw]=1
    y=np.zeros(totalrows*2)
    y[best]=yb
    y[worst]=yw
    return X,y

In [7]:
def estimateparams(X,questions,nb,nw,w0=[],tol=.0001,maxiter=100):
    tol=.001
    val=1
    [numofq,nitems]=X.shape
    if len(w0)<nitems:
        w=np.zeros(nitems)    
    else:
        w=w0.copy()
    iternum=0
    try:
        while val>tol and iternum<maxiter:
            zb=np.sum(np.exp(w[questions]),1)
            zw=np.sum(np.exp(-w[questions]),1)
            Pb=((1/zb*X.T).T*np.exp(w))#broadcasting
            Pw=((1/zw*X.T).T*np.exp(-w))#broadcasting
            gradb=nb-np.sum(Pb,0)
            gradw=-nw+np.sum(Pw,0)
            grad=-(gradb+gradw)
            Hb=np.dot(Pb.T,Pb)
            Hbd=-np.sum(Pb*(1-Pb),0)
            Hb+=-np.diag(np.diag(Hb))+np.diag(Hbd)
            Hw=np.dot(Pw.T,Pw)
            Hwd=-np.sum(Pw*(1-Pw),0)
            Hw+=-np.diag(np.diag(Hw))+np.diag(Hwd)
            H=-(Hb+Hw)
            gradl=grad[:-1]
            Hl=H[:-1,:-1]
            w[:-1]=w[:-1]-np.linalg.solve(Hl,gradl)
            val=np.dot(gradl,np.linalg.solve(Hl,gradl))
            iternum+=1
    except np.linalg.linalg.LinAlgError:
        return w,np.eye(nitems-1)
    return w,np.linalg.inv(Hl)

In [8]:
def negLogLik(w,questions,best,worst):
    return -(np.sum(np.log(1/np.sum(np.exp(w[questions]),1)*np.exp(w[best])))+
             np.sum(np.log(1/np.sum(np.exp(-w[questions]),1)*np.exp(-w[worst]))))

    

In [9]:
def run100trials(samplescheme,filename):
    n=100
    numresp=1000
    nofqsbot=12
    choicedata=np.zeros((n*numresp*nofqsbot,11),dtype=np.int)
    nitems=120
    batchsize=20
    numrows=numresp//batchsize
    allbetas=np.zeros((numrows*n,nitems+2))
    for i in range(n):
        betas,allquestions,allbest,allworst,people=simulation(
            numresp,fixedsparse,batchsize,'HB_120items.csv','20itemMD_MD20_Design.csv',0)
        rowstart=i*numresp*nofqsbot
        rowend=(i+1)*numresp*nofqsbot
        choicedata[rowstart:rowend,0]=i+1
        for j,person in enumerate(people):
            choicedata[rowstart+j*nofqsbot:rowstart+(j+1)*nofqsbot,1]=person
            choicedata[rowstart+j*nofqsbot:rowstart+(j+1)*nofqsbot,2]=j
            choicedata[rowstart+j*nofqsbot:rowstart+(j+1)*nofqsbot,3]=[x for x in range(1,nofqsbot+1)]
        choicedata[rowstart:rowend,4:9]=allquestions+1
        choicedata[rowstart:rowend,9]=allbest+1
        choicedata[rowstart:rowend,10]=allworst+1
        allbetas[i*numrows:(i+1)*numrows,0]=i+1
        allbetas[i*numrows:(i+1)*numrows,1]=[x for x in range(20,numrows*20+20,20)]
        allbetas[i*numrows:(i+1)*numrows,2:]=betas
    choicedata_df=pd.DataFrame(choicedata)
    choicedata_df.columns=["Iter","original_ids","RespIDNum","QNum",
                           "Choice1","Choice2","Choice3","Choice4","Choice5",
                           "Best","Worst"]
    allbetas_df=pd.DataFrame(allbetas)
    colname=["Iter","RespNum"]
    for i in range(nitems):
        colname+=["Item_"+str(i+1)]
    allbetas_df.columns=colname
    choicedata_df.to_csv("Results/choice_data-"+filename)
    allbetas_df.to_csv("Results/bhist-"+filename)

In [47]:
#filename="fixed_express-120v20.csv"
run100trials(fixedexpress,filename)

In [27]:
filename="fixed_sparse-120v30.csv"
run100trials(fixedsparse,filename)

In [29]:
filename="fixed_sparse-120v20.csv"
run100trials(fixedsparse,filename)

In [224]:
np.mean(np.random.dirichlet(np.ones(18*1000)))

5.5555555555555924e-05

In [18]:
t=time.time()
betas,allquestions,allbest,allworst,people=simulation(1000,fixedexpress,20,'HB_120items.csv','30itemMD_MD30_Design.csv',10)
time.time()-t

ValueError: need more than 4 values to unpack

In [36]:
(np.random.rand()<.1)

array([-0.20859403, -1.88179221, -0.80093614,  0.31997618,  0.48053349,
        0.5843839 , -0.07022752, -1.12089193,  0.23116713,  0.34531458,
        0.49603144, -0.39426293, -1.08377876, -0.45156412,  0.62991068,
        0.31716553,  0.63055001,  0.56938473, -0.57483069,  0.25082672,
        0.61987033, -0.13226487,  0.27372134,  0.33642631, -2.64641515,
       -0.3306664 , -0.20300979, -0.17841469, -0.11343263,  0.12059032,
       -0.09477682,  0.29002589,  0.55104746, -0.46624373, -2.32734791,
        0.05230717,  0.98823041, -0.35586035, -0.2833065 , -0.11835683,
        0.14366094,  0.05625812,  0.86370855,  0.56748716, -0.43489025,
        0.313019  , -0.69366764, -0.6873674 ,  1.25400446, -0.49961911,
        0.63528861, -1.49625693, -0.34811686,  0.85654422,  1.1343112 ,
       -0.45287853,  0.27504613,  0.53302838,  0.21096076,  0.69053469,
        0.04673277, -0.35263293,  1.25980274,  0.00743529, -0.69786084,
       -1.27126124,  0.08766714, -0.49198188, -0.89974532, -0.27

In [10]:
t=time.time()
X,allquestions,allbest,allworst=simulation(500,,500,'HB_120items.csv','30itemMD_MD30_Design.csv',0)
time.time()-t

0.8102290630340576

In [14]:
[numofq,nitems]=X.shape

In [16]:
subsamp=int(numofq*.25)
sub=np.random.choice([x for x in range(0,numofq)],subsamp)
tempX=X[sub]
tempb=allbest[sub]
tempw=allworst[sub]
tempq=allquestions[sub]

In [17]:
sub

array([7379, 6663, 1683, ..., 1717,  220,   24])

In [25]:
tempw[0]

105

In [26]:
allworst[7379]

105

In [14]:
t=time.time()
[numofq,nitems]=X.shape
Xb=np.zeros((numofq,nitems))
Xb[range(numofq),allbest]=1
nb=np.sum(Xb,0)
Xw=np.zeros((numofq,nitems))
Xw[range(numofq),allworst]=1
nw=np.sum(Xw,0)
time.time()-t

0.010955095291137695

In [22]:
np.bincount(allquestions.flatten())

array([375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375,
       375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375,
       375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375,
       375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375,
       375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375,
       375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375,
       375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375,
       375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375,
       375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375, 375,
       375, 375, 375])

In [19]:
t=time.time()
nb=np.bincount(allbest)
nw=np.bincount(allworst)
time.time()-t

0.00047206878662109375

In [17]:
np.allclose(nw,nw2)

True

In [39]:
t=time.time()
beta,cov=estimateparams(X,allquestions,nb,nw)
time.time()-t

1.0672240257263184

In [80]:
def bayesBootstrap(X,questions,allbest,allworst,numdraws=1):
    [numofq,nitems]=X.shape
    draws=np.zeros((numdraws,nitems))
    [numofq,nitems]=X.shape
    Xb=np.zeros((numofq,nitems))
    Xb[range(numofq),allbest]=1
    Xw=np.zeros((numofq,nitems))
    Xw[range(numofq),allworst]=1
    #w=estimateparamsbayes(X,questions,Xb,Xw,w0=[],weights=np.ones(numofq))
    for i in range(numdraws):
        #weights=np.random.dirichlet(np.ones(numofq))*numofq
        weights=np.random.exponential(1,numofq)
        draws[i]=estimateparamsbayes(X,questions,Xb,Xw,w0=[],weights=weights)
    return draws

In [81]:
def estimateparamsbayes(X,questions,Xb,Xw,w0=[],weights=[],tol=.0001,maxiter=100):
    tol=.001
    val=1
    [numofq,nitems]=X.shape
    if len(w0)<nitems:
        w=np.zeros(nitems)    
    else:
        w=w0.copy()
    if len(weights)<numofq:
        weights=np.ones(numofq)
    iternum=0
    nb=np.sum(Xb.T*weights,1)
    nw=np.sum(Xw.T*weights,1)
    try:
        while val>tol and iternum<maxiter:
            if iternum:
                w[:-1]=w[:-1]-Hinvgradl
            zb=np.sum(np.exp(w[questions]),1)
            zw=np.sum(np.exp(-w[questions]),1)
            Pb=((1/zb*X.T).T*np.exp(w))#broadcasting
            Pw=((1/zw*X.T).T*np.exp(-w))#broadcasting
            gradb=nb-np.sum((Pb.T*weights).T,0)
            gradw=-nw+np.sum((Pw.T*weights).T,0)
            grad=-(gradb+gradw)
            Hb=np.dot(weights*Pb.T,Pb)
            Hbd=-np.sum((Pb.T*weights).T*(1-Pb),0)
            Hb+=-np.diag(np.diag(Hb))+np.diag(Hbd)
            Hw=np.dot(weights*Pw.T,Pw)
            Hwd=-np.sum((Pw.T*weights).T*(1-Pw),0)
            Hw+=-np.diag(np.diag(Hw))+np.diag(Hwd)
            H=-(Hb+Hw)
            gradl=grad[:-1]
            Hl=H[:-1,:-1]
            Hinvgradl=np.linalg.solve(Hl,gradl)
            val=np.dot(gradl,Hinvgradl)
            iternum+=1
    except np.linalg.linalg.LinAlgError:
        return w
    return w

In [75]:
def negLogLikBayes(w,questions,best,worst,weights):
    return -(np.sum(weights*np.log(1/np.sum(np.exp(w[questions]),1)*np.exp(w[best])))+
             np.sum(weights*np.log(1/np.sum(np.exp(-w[questions]),1)*np.exp(-w[worst]))))

In [82]:
t=time.time()
np.random.seed(105)
drawbeta4=bayesBootstrap(X,allquestions,allbest,allworst,100)
time.time()-t

29.676923036575317

In [92]:
i=45
plt.hist(drawbeta[:,i],bins=50,alpha=.5)
plt.hist(drawbeta2[:,i],bins=50,alpha=.5)
plt.hist(drawbeta4[:,i],bins=50,alpha=.5)
plt.show()

In [88]:
np.diag(np.cov(drawbeta2.T))/np.diag(np.cov(drawbeta.T))

array([ 2.9009285 ,  2.84153861,  3.1380559 ,  3.0936153 ,  2.61088036,
        2.85467994,  2.3000842 ,  3.30411487,  3.78943652,  2.21121796,
        2.28610091,  2.75855298,  2.50640793,  3.22157588,  2.60994759,
        2.11109416,  2.63475893,  3.04220898,  2.78382252,  3.01697189,
        2.30876494,  2.89301505,  3.20217522,  3.19137359,  3.3933343 ,
        2.2495875 ,  2.97405055,  2.76121995,  2.84167685,  2.55096602,
        3.42533864,  2.18191481,  3.01361518,  1.91343535,  3.27194031,
        2.47878249,  2.62852595,  2.57245712,  2.82926515,  2.8761957 ,
        2.70638723,  2.67446146,  2.98778703,  2.74500853,  2.28656352,
        4.22547377,  2.58524419,  2.62009297,  2.72368692,  3.52838103,
        2.61673584,  2.6714492 ,  2.67954869,  2.56232461,  3.02061072,
        3.89361569,  2.20762918,  3.34041182,  3.2618624 ,  2.69611343,
        3.54647674,  2.58490112,  2.37046438,  2.84531339,  2.65682763,
        2.67924638,  3.4504129 ,  3.79846199,  3.21405546,  2.68

In [89]:
np.diag(np.cov(drawbeta4.T))/np.diag(np.cov(drawbeta.T))

array([  5.85073242,   6.28979981,   8.37848945,   6.94537905,
         6.69320806,   7.40219626,   5.98958618,   7.55817933,
         9.2963098 ,   6.07660179,   5.45320779,   6.85655985,
         6.86785658,   9.31825502,   5.61419323,   5.98068716,
         7.08891879,   9.64383145,   5.80677878,   6.07211501,
         6.38441641,   6.69287194,   6.53965536,   7.96718863,
         6.94713751,   5.58309705,   8.18930153,   7.22165648,
         7.55756398,   5.96362238,   7.11650367,   7.03924644,
         8.55570374,   6.11449029,   7.94622896,   5.89135341,
         7.23293622,   5.72249078,   6.93405524,   7.16361648,
         6.32173252,   6.20297284,   7.31766969,   6.74662599,
         6.52774444,   9.62223498,   7.24110259,   6.79587048,
         5.99077464,   7.8807248 ,   7.71951437,   6.64806379,
         5.9993897 ,   6.79444098,   7.77482327,   9.5955848 ,
         5.63003652,   7.74358155,   8.07540254,   7.80100813,
         7.47171474,   6.24739092,   5.5030783 ,   7.67

array([ 0.14330843,  0.17552696,  0.18539527,  0.16417409,  0.14232249,
        0.18569621,  0.14552542,  0.16385987,  0.23063395,  0.15891115,
        0.14825714,  0.17773712,  0.17015696,  0.18198421,  0.14061992,
        0.15977119,  0.17146158,  0.19157827,  0.12955605,  0.12843327,
        0.15060367,  0.15890357,  0.16555681,  0.17785695,  0.18284256,
        0.13282111,  0.17683296,  0.18681357,  0.16682857,  0.1332859 ,
        0.15121658,  0.17223127,  0.17498468,  0.1486135 ,  0.18189858,
        0.13860361,  0.16791912,  0.1546725 ,  0.14737932,  0.14579706,
        0.16528384,  0.16117271,  0.17245879,  0.14448107,  0.15699122,
        0.20021327,  0.19263609,  0.18969645,  0.14070885,  0.13783123,
        0.17631595,  0.17619313,  0.15636607,  0.14298475,  0.1626591 ,
        0.1831898 ,  0.1351011 ,  0.16199476,  0.17451772,  0.1921313 ,
        0.17479646,  0.16271761,  0.16318134,  0.17549009,  0.16179619,
        0.15764841,  0.13466458,  0.17337613,  0.224876  ,  0.18

In [34]:
for i in [34,7,11,0,35,88,53,48]:
    plt.hist(drawbeta[:,i],bins=50,alpha=.5)
    plt.hist(dbeta[:,i],bins=50,alpha=.5)
    plt.savefig('a100item'+str(i+1))
    plt.close()

0.03612085890769959

In [164]:
w=estimateparamsbayes(X,allquestions,Xb,Xw,w0=[],weights=np.ones(numofq))

In [234]:
np.random.seed(1434)
weights1=np.random.dirichlet(np.ones(numofq))
np.random.seed(1434)
weights2=np.random.exponential(1,numofq)

In [235]:
w1=estimateparamsbayes(X,allquestions,Xb,Xw,w0=w,weights=weights1)
w2=estimateparamsbayes(X,allquestions,Xb,Xw,w0=w,weights=weights2)
w4=estimateparamsbayes(X,allquestions,Xb,Xw,w0=[],weights=weights1)
w5=estimateparamsbayes(X,allquestions,Xb,Xw,w0=[],weights=weights2)
negLogLikBayes(w,allquestions,allbest,allworst,weights1*numofq)
negLogLikBayes(w1,allquestions,allbest,allworst,weights1*numofq)
negLogLikBayes(w2,allquestions,allbest,allworst,weights1*numofq)
negLogLikBayes(w4,allquestions,allbest,allworst,weights1*numofq)
negLogLikBayes(w5,allquestions,allbest,allworst,weights1*numofq)

In [ ]:
utilsall=data.get_values()[:,2:]

In [8]:
MDDesign =pd.read_csv('30itemMD_MD30_Design.csv')#get the files
design=MDDesign.get_values()[:,2:]#changes to a numpy array
numver=max(MDDesign['Version'])#number of versions
qperset=max(MDDesign['Set'])

In [7]:
designm=MDDesign.get_values()

In [14]:
np.save('design',designm)

In [9]:
design1=designm[:,2:]
numver1=max(designm[:,0])#number of versions
qperset1=max(designm[:,1])

In [12]:
np.allclose(design1,design)

True

In [ ]:
# exp 5.198677062988281
#5.124932050704956

In [43]:
t=time.time()
dbeta=np.zeros((1000,300))
dbeta[:,:-1]=np.random.multivariate_normal(beta[:-1], cov,1000)
time.time()-t

0.04565596580505371

In [24]:
order

[108,
 21,
 46,
 12,
 3,
 110,
 23,
 89,
 67,
 80,
 28,
 47,
 72,
 20,
 94,
 1,
 79,
 9,
 0,
 99,
 93,
 6,
 61,
 45,
 29,
 95,
 118,
 38,
 92,
 19,
 76,
 31,
 106,
 39,
 85,
 104,
 40,
 34,
 52,
 10,
 24,
 87,
 4,
 13,
 66,
 114,
 73,
 14,
 120,
 27,
 25,
 113,
 115,
 111,
 64,
 2,
 119,
 60,
 88,
 63,
 97,
 117,
 82,
 16,
 49,
 103,
 96,
 101,
 17,
 90,
 7,
 50,
 78,
 22,
 109,
 41,
 32,
 5,
 15,
 35,
 30,
 43,
 55,
 58,
 42,
 77,
 91,
 81,
 54,
 56,
 33,
 105,
 8,
 100,
 75,
 102,
 62,
 26,
 59,
 48,
 65,
 36,
 69,
 86,
 98,
 68,
 84,
 71,
 116,
 107,
 51,
 74,
 112,
 37,
 70,
 18,
 53,
 83,
 57,
 11,
 44,
 79,
 22,
 36,
 78,
 7,
 106,
 57,
 118,
 6,
 55,
 19,
 34,
 107,
 46,
 101,
 96,
 114,
 40,
 60,
 62,
 2,
 63,
 26,
 110,
 88,
 64,
 33,
 12,
 90,
 10,
 109,
 61,
 82,
 102,
 32,
 108,
 92,
 67,
 72,
 18,
 111,
 84,
 87,
 120,
 0,
 73,
 47,
 116,
 97,
 4,
 5,
 27,
 91,
 13,
 50,
 113,
 51,
 31,
 24,
 49,
 85,
 94,
 75,
 105,
 104,
 112,
 3,
 83,
 35,
 59,
 56,
 44,
 29,
 17,
 45,


In [292]:
data=pd.read_csv('HB_120items.csv')
utilsall=data.get_values()[:,2:]
realmean=np.mean(utilsall,0)

In [15]:
data=pd.read_csv('HB_120items.csv')
utilsall=data.get_values()[:,2:]

In [16]:
np.save('120util',utilsall)

In [192]:
t=time.time()
bz,sez=bayesBootstrapz(n,nb,nw,10)
time.time()-t

19.807085037231445

In [17]:
MDDesign =pd.read_csv(designfile)#get the files
utilsall=data.get_values()[:,2:]

93

In [ ]:
def simulation(nrespondents,k,samplescheme,batchsize,itemfile,designfile,seed=0):#doesnot work
    if seed>0:
        np.random.seed(seed)#for reproducibility
    data=pd.read_csv(itemfile)#
    MDDesign =pd.read_csv(designfile)#get the files
    utilsall=data.get_values()[:,2:]#utils of bots
    [numreal,nitems]=utilsall.shape#
    design=MDDesign.get_values()[:,2:]#changes to a numpy array
    numver=max(MDDesign['Version'])#number of versions
    qperset=max(MDDesign['Set'])#questions per set
    itemsperq=design.shape[1]#items per questions
    itemsperset=np.max(design)#items per set
    people=np.random.choice([x for x in range(0,numreal)],nrespondents)#choice the bots
    cov=np.eye(nitems)
    totalrows=qperset*itemsperq#rows
    allquestions=np.zeros((nrespondents*qperset,itemsperq),dtype=np.int)
    allbest=np.zeros(nrespondents*qperset,dtype=np.int)
    allworst=np.zeros(nrespondents*qperset,dtype=np.int)
    X=np.zeros((nrespondents*qperset,nitems))
    w=np.zeros(nitems)
    numrows=nrespondents//batchsize
    betas=np.zeros((numrows,nitems))
    questions=fixedexpress(nitems,batchsize,design[:(qperset)*(batchsize)]-1)
    qbatch=qperset*batchsize
    for iternum in range(0,numrows):
        for i in range(batchsize):
            util=utilsall[people[iternum*batchsize+i],:]
            print(util.shape)
            bestitems,worstitems=fillsurvey(questions[i*qperset:(i+1)*qperset],util)
            for j,bitem in enumerate(bestitems):
                allbest[iternum*qbatch+i*qperset+j]=bitem
            for j,witem in enumerate(worstitems):
                allworst[iternum*qbatch+i*qperset+j]=witem
        for i,oneq in enumerate(questions):
            X[iternum*qperset+i,oneq]=1
        allquestions[iternum*qbatch:(iternum+1)*qbatch]=questions
        #betas[iternum]=w
        vernum=((iternum+1)*batchsize)%numver
        respdesign=design[qperset*vernum:(qperset)*(vernum+batchsize)]-1
        Xtemp=X[:(iternum+1)*qbatch]
        qtemp=allquestions[:(iternum+1)*qbatch]
        besttemp=allbest[:(iternum+1)*qbatch]
        worsttemp=allworst[:(iternum+1)*qbatch]
        questions=samplescheme(Xtemp,qtemp,besttemp,worsttemp,k,batchsize,respdesign)
        betas[iternum]=estimateparams(Xtemp,qtemp,besttemp,worsttemp)
    return betas,allquestions,allbest,allworst,people